# MGSC 404 Foundations of Decision Analytics - Fall 2020

# Case Study

## Team Members: 

Jonathan Steinberg 260924571 jonathan.steinberg@mail.mcgill.ca
Tyra David 260869114 tyra.david@mail.mcgill.ca
Julien Desautels 260585827 julien.desautels@mail.mcgill.ca


# **Section I: Introduction**

American Office Systems Inc., established in 1939, is a manufacturer of office equipment based in San Francisco. In the 1950s, when computers began to erode its market for bookkeeping machines, the company diversified and began to manufacture IBM and Macintosh adapters in order to tap-into the copy machine market. The goal of this case study is to optimize American Office Systems’ profit for the year 1996 in the face of many production, marketing, inventory, borrowing, and cash flow constraints. The findings of this study show that, in order to maximize profit, more IBM adapters should be produced and sold than Macintosh adapters. Furthermore, the company will not elect to borrow the any of the 750,000 dollars from the line-of-credit. Overall, the optimal solution will result in a profit of 13,031,411.85 dollars at the end of the fourth quarter of 1996.



# **Section II: Model**


**Assumptions**

Our model relies on the following assumptions:

1. American Office Systems, Inc. runs 2 work shifts of 8 hours every day for 20 days per month. Each quarter is made of 3 months therefore we made our full capacity the number of adapters for IBM and MAC produced each hour multiplied by the hours worked in a quarter. 
2. Forecasted sales are accurate 
3. We immediately pay the interest for borrowing in the previous quarter at the end of the following quarter and pay off all credit line from last quarter. Credit Line must all be paid for by Q4.
4. Using a linear model is an appropriate approximation for an optimal strategy and further refinement will be done later. 
5. This model optimizes the accounting profit instead of the present and future cash flows of the operations. We opted for that sort of model because it depicts better the reality of the company and we found that, with the cash flow based models, the ending inventory of the last quarter was often too low to prepare for next year.
6. The fixed costs of 3 million dollars are for the whole company and only a portion is attributable to the adapters. We decided that 250,000 would be a good starting point.
7. The leftover Mac sales that are not paid for each quarter are also sold to collection agency at 50% of sales value three quarters after initial sale.
8.Same 6% reinvestment rate applies for Q1-2 of 1997 and becomes their discount rate.
9. Model should account for Q4 sales received in Q1-2 of 1997 and cash received from previous year's sales (1995 Q3 and Q4).
10. Average inventories follow a linear model using beginning and end inventories by quarter and are used to estimate stock carrying costs and % was taken off variable cost of each type of adapter (as is usual in accounting).
11.Temporal frame of reference for value of all cash flows is their PV at end of Q4 1996.
12. Marketing spend is determined quarterly and is separated by product line (IBM and Mac) for optimal sales targeting of either product according to needs.
13. Ending inventory/Minimum production calculation: for expected sales of Q1-3 used regular + Marketing sales instead of max base expected sales besides for the last quarter Q4 since it is explicitly mentioned to use max expected sales.
14. Thus, the constraint for Q1-3 of “production plus the end-of-period inventory for the adapters should be at least 10% larger than the estimated sales for the next period” was taken literally as production Q1 + Ending inventory Q1 >= 1.1(regular sales Q2 + Marketing sales Q2).
15. Carrying costs are assumed at the end of each quarter, none for 1997 since it does not pertains to next year’s activities just as beginning stock of 1996 included in 1996.
16. Depreciation not factored in model since never paid for.

In [ ]:
from gurobipy import *
from datascience import *
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import math
import IPython.display



ModuleNotFoundError: ignored

### **Step 1: Creating a Gurobi object**

In [ ]:
model = Model('case study')  

NameError: ignored

### **Step 2: Defining Decision Variables**

**Pi** = Production for IBM adapters in units of adapters, in quarters i, where i = 1,2,3,4             

**Pm** =  Production for MAC adapters in units of adapters, in quarters i, where i = 1,2,3,4   

**Si** = Total Sales (including Marketing) for IBM adapters in units of adapters, in quarters i, where i = 1,2,3,4   

**Sm** = Total Sales (including Marketing) for MAC adapters in units of adapters, in quarters i, where i = 1,2,3,4 

**Ri** = Regular Sales for IBM adapters in units of adapters, in quarters i, where i = 1,2,3,4

**Rm** = Regular Sales for MAC adapters in units of adapters, in quarters i, where i = 1,2,3,4

**Mi** = Marketing for IBM adapters in units of additional adapter units sold due to marketing activities, in quarters i, where i = 1,2,3,4   

**Mm** = Marketing for MAC adapters in units of additional adapter units sold due to marketing activities, in quarters i, where i = 1,2,3,4  <br />

**B  =** Amount borrowed from the line of credit in units of dollars, in quarters i, where i = 1,2,3,4 <br />

**Ei** = Ending inventory for IBM adapters in units of adapters, in quarters i, where i = 1,2,3,4   

**Em** = Ending inventory for MAC adapters in units of adapters, in quarters i, where i = 1,2,3,4   

**CF** =Cash flow at the end of the quarter in units of dollars, in quarters i, where i = 1,2,3,4 <br />

**F** =Cash flow at the end of the quarter in units of dollars, in quarters i, where i = 1,2 of 1997  


In [ ]:
#initilizing empty dictionaries
Pi = {}  
Pm = {} 
Si = {} 
Sm = {} 
Mi = {} 
Mm = {} 
B = {} 
Ei = {}
Em = {} 
CF={}
F= {}
Ri={}
Rm={}

# Create decisions variable
for i in np.arange(4):  
    Pi[i] = model.addVar(lb=0)
    Pm[i] = model.addVar(lb=0)
    Si[i] = model.addVar(lb=0)
    Sm[i] = model.addVar(lb=0)
    Mi[i] = model.addVar(lb=0)
    Mm[i] = model.addVar(lb=0)
    B[i] = model.addVar(lb=0)
    CF[i] = model.addVar(lb=0)
    Ei[i] = model.addVar(lb=0)
    Em[i] = model.addVar(lb=0)
    Ri[i] = model.addVar(lb=0)
    Rm[i] = model.addVar(lb=0)

for i in np.arange(2):    
    F[i] = model.addVar(lb=0)
    # By default Gurobi assumes that the lower bound of a variable lb=0, not minus infinity.

### **Step 3: Constant Parameters**

**price_ibm** = the selling price for IBM adapters  

**price_mac** = the selling price for MAC adapters  

**VC_ibm** =  the variable costs for IBM adapters 

**VC_mac** =  the variable costs for MAC adapters 

**beginning_inventory_ibm** =  the beginning inventory for IBM adapters at Q1  

**beginning_inventory_mac** =  the beginning inventory for MAC adapters at Q1 

**cost_advertising** =  the cost to sell one additional unit of either an IBM or MAC adapter  

**interest_rate_LOC** = The annual interest rate for the line of credit 

**interest_rate_bonds** = The annual interest rate for the government securities 

**fixed_costs** = The fixed costs per quarter

**carry_cost_month** = The monthly carry cost of inventory


In [ ]:
# constant parameters  
price_ibm = 175
price_mac = 200
VC_ibm = 100
VC_mac = 110
beginning_inventory_ibm = 10000
beginning_inventory_mac = 5000
cost_advertising = 10
interest_rate_LOC = 0.16
interest_rate_bonds = 0.06
fixed_costs = 250000
carry_cost_month = 0.01

### **Step 4: Constraints**

**Maximum Production IBM Constraint:** We included production constraints on IBM adapters of ast most 28800 units for the first three quarters, since our calculation assumed two shifts of 8 hours a day, 20 production days a month, 3 months a quarter, and at most 30 IBM adapters per hour. Our last quarter has a production constraint of at most 31680 as that accounts for the 10% increase in capacity by the beginning of the first quarter.   
  
**Maximum Production MAC Constraint:** We included production constraints on MAC adapters of at most 9600 units for the first three quarters, since our calculation assumed two shifts of 8 hours a day, 20 production days a month, 3 months a quarter, and at most 10 MAC adapters per hour. Our last quarter has a production constraint of at most 10560 as that accounts for the 10% increase in capacity by the beginning of the first quarter. 

**Minimum Production IBM Constraint:** We included production constraints on IBM adapters of at least 2400 units for each of the four quarters, since our calculation assumed 20 production days a month, 3 months a quarter, and at least 40 IBM adapters produced per production day. This constraint was established in order to maintain continuity in the production process, which can lower the risk of equipment malfunction. 

**Minimum Production MAC Constraint:** We included production constraints on MAC adapters of at least 1800 units for each of the four quarters, since our calculation assumed 20 production days a month, 3 months a quarter, and at least 30 MAC adapters produced per production day. This constraint was established in order to maintain continuity in the production process, which can lower the risk of equipment malfunction. 

**Ending Inventory IBM:** The ending inventory of IBM adapters is calculated as the starting inventory in the current quarter (which is equivalent to the ending inventory from the previous quarter) plus the production done during the current quarter minus the total sales made during the current quarter. Note that for IBM adapters, 1995 inventory ended with 10000 adapters, and therefore 1996 starts with this quantity. 

**Ending Inventory MAC:** The ending inventory of IBM adapters is calculated as the starting inventory in the current quarter (which is equivalent to the ending inventory from the previous quarter) plus the production done during the current quarter minus the total sales made during the current quarter. Note that for IBM adapters, 1995 inventory ended with 5000 adapters, and therefore 1996 starts with this quantity. 

**Inventory Management Constraint IBM:** We included inventory management constraints for IBM adapters. Specifically, the production of IBM adapters in a partiular quarter, plus the end-of-period iventory of IBM adapters in the same quarter, must be 10% larger than the estimated sales of IBM adapters for the next quarter. There are only three constraints in this section, for quarters 1, 2, and 3. There is no constarint for quarter 4 because the next quarter after the fourth quarter of 1996 would be the first quarter of 1997, which is outside of the scope of this optimization problem.  This constraint was established to provide a safety cushion to help prevent shortages of IBM
adapters.   

**Inventory Management Constraint MAC:** We included inventory management constraints for MAC adapters. Specifically, the production of MAC adapters in a partiular quarter, plus the end-of-period iventory of MAC adapters in the same quarter, must be 10% larger than the estimated sales of MAC adapters for the next quarter. There are only three constraints in this section, for quarters 1, 2, and 3. There is no constarint for quarter 4 because the next quarter after the fourth quarter of 1996 would be the first quarter of 1997, which is outside of the scope of this optimization problem. This constraint was established to provide a safety cushion to help prevent shortages of MAC
adapters.

**Fourth Quarter IBM Inventory and Production Constraint:** We included fourth quarter inventory and production constraints on IBM adapters. Specifically, the inventory level plus production of IBM adapters at the end of the fourth quarter of 1996 should be at least twice the maximum expected sales of IBM adapters for that quarter. Since the maximum expected sales of IBM adapters in the fourth quarter of 1996 is 12000, the inventory level plus production of IBM adapters at the end of the fourth quarter of 1996 should be at least 24000, which is 2 multiplied by 12000. This constraint was established to provide a smooth transition to 1997. 

**Fourth Quarter MAC Inventory and Production Constraint:** We included fourth quarter inventory and production constraints on MAC adapters. Specifically, the inventory level plus production of MAC adapters at the end of the fourth quarter of 1996 should be at least twice the maximum expected sales of MAC adapters for that quarter. Since the maximum expected sales of MAC adapters in the fourth quarter of 1996 is 2400, the inventory level plus production of MAC adapters at the end of the fourth quarter of 1996 should be at least 4800, which is 2 multiplied by 2400. This constraint was established to provide a smooth transition to 1997. 

**Marketing Constraint IBM:** We included a marketing constraint for IBM adapters. Specifically, the additional units sold of IBM adapters resulting from the marketing activities in a particular quarter should account for at least a third of total IBM adapters sold in that quarter. This constraint was established to allow the company to spend money on marketing and to ensure that marketing activities are considered effective. 

**Marketing Constraint MAC:** We included a marketing constraint for MAC adapters. Specifically, the additional units sold of MAC adapters resulting from the marketing activities in a particular quarter should account for at least a third of total MAC adapters sold in that quarter. This constraint was established to allow the company to spend money on marketing and to ensure that marketing activities are considered effective. 

**Marketing Demand Constraint IBM:** We included a marketing demand constraint for IBM adapters. Specifically, the sales of IBM adapters in any quarter should be below or equal to the maximum forecasted demand of IBM adapters plus the additional demand in units of IBM adapters generated by marketing activities in the same quarter. The maximum forcasted demand of IBM adapters for quarter 1, 2, 3, and 4 is 9000, 10000, 11000, and 12000 respectively. This constraint was established to ensure that our model does not maximize sales beyond our ability.

**Marketing Demand Constraint MAC:** We included a marketing demand constraint for MAC adapters. Specifically, the sales of MAC adapters in any quarter should be below or equal to the maximum forecasted demand of MAC adapters plus the additional demand in units of MAC adapters generated by marketing activities in the same quarter. The maximum forcasted demand of IBM adapters for quarter 1, 2, 3, and 4 is 1800, 2000, 2200, and 2400 respectively. This constraint was established to ensure that our model does not maximize sales beyond our ability.

**Maximum Borrowing Constraint:**
We included a maximum borrowing constaint for the line of credit. Specifically, the individual amounts borrowed from the line of credit in units of dollars in each quarter cannot exceed 750,000, since this is the amount that the credit line is worth as a whole. Note that the borrowing principal and interest is paid back the following quarter and a new borrowing is made. 

**Cash Flow Constraints:** We included constraints to define our cash flows appropriately. The quarterly cash flow results from the sales of IBM and MAC adapters, which are calculated using the selling price, and keeping in mind the collection percentages per quarter. In addition, the production costs of both adapters are subtracted as well as the marketing costs for the extra adapters sold through marketing initiatives. Subtracted is then the holding cost where the inventory level is taken as the average inventory throughout that quarter. Subtracted again is the fixed cost per quarter. In terms of borrowing, the principal of the borrowing from the quarter before is payed back the next quarter with interest, and a new borrowing is made. This ensures that borrowing will never be above the $750,000 limit and leaves the flexibility for the company to vary their amount borrowed month to month as a line of credit acts as a pseudo chequing account. The borrowing payed back is not applicable for the initial cash flow, as there has been no costs associated with borrowing yet. Note that all net profit is then invested in government securities and the principal invested plus interest is received the following quarter, which is shown in the objective function. 

**Cash Flow First Two Quarters of 1997:** Collection is still being made in 1997 for adapters that are sold in 1996. Therefore, the incoming cash flows of 1997 include the collection. The borrowing from 1996 is also being paid back, which is subtract from this cash flow. All of the 1997 cash flows are discounted back 1/2 quarters using the interest rate of the government securities. Also note that in 1997, no new borrowing is made, only borrowings are paid back from the previous year. 

**Non-Negative Cash Flows:** These constraints limit the net profit with borrowing to be strictly positive. The addition of the borrowing therefore results in more flexibility to have negative net profit, which might produce more sales, and will ultimately generate greater profit after the year. 

**Regular Sales Limited by Forecasted Demand IBM:** These constraints explain how the company will be ale to sell any number of IBM adapters up to the maximum forecasted sales amounts shown in the table 4.17 of the case study. 

**Regular Sales Limited by Forecasted Demand Mac:** These constraints explain how the company will be ale to sell any number of Mac adapters up to the maximum forecasted sales amounts shown in the table 4.17 of the case study. 

In [ ]:
# maximum production ibm constraint
con_max_production_ibm={}
con_max_production_ibm[0]=model.addConstr(Pi[0]<=28800)
con_max_production_ibm[1]=model.addConstr(Pi[1]<=28800)
con_max_production_ibm[2]=model.addConstr(Pi[2]<=28800)
con_max_production_ibm[3]=model.addConstr(Pi[3]<=31680)

# maximum production mac constraint
con_max_production_mac={}
con_max_production_mac[0]=model.addConstr(Pm[0]<=9600)
con_max_production_mac[1]=model.addConstr(Pm[1]<=9600)
con_max_production_mac[2]=model.addConstr(Pm[2]<=9600)
con_max_production_mac[3]=model.addConstr(Pm[3]<=10560)

In [ ]:
# minimum production ibm constraint
con_min_production_ibm={}
con_min_production_ibm[0]=model.addConstr(Pi[0]>=2400)
con_min_production_ibm[1]=model.addConstr(Pi[1]>=2400)
con_min_production_ibm[2]=model.addConstr(Pi[2]>=2400)
con_min_production_ibm[3]=model.addConstr(Pi[3]>=2400)

# minimum production mac constraint
con_min_production_mac={}
con_min_production_mac[0]=model.addConstr(Pm[0]>=1800)
con_min_production_mac[1]=model.addConstr(Pm[1]>=1800)
con_min_production_mac[2]=model.addConstr(Pm[2]>=1800)
con_min_production_mac[3]=model.addConstr(Pm[3]>=1800)

In [ ]:
# inventory management constraint ibm
estimated_sale_ibm={}
estimated_sale_ibm[0]=model.addConstr(Pi[0] +Ei[0] >= 1.1*Si[1])
estimated_sale_ibm[1]=model.addConstr(Pi[1] +Ei[1] >= 1.1*Si[2])
estimated_sale_ibm[2]=model.addConstr(Pi[2] +Ei[2] >= 1.1*Si[3])

# inventory management constraint mac
estimated_sale_mac={}
estimated_sale_mac[0]=model.addConstr(Pm[0] +Em[0] >= 1.1*Sm[1])
estimated_sale_mac[1]=model.addConstr(Pm[1] +Em[1] >= 1.1*Sm[2])
estimated_sale_mac[2]=model.addConstr(Pm[2] +Em[2] >= 1.1*Sm[3])

In [ ]:
# ending inventory ibm 
ending_inventory_ibm={}
ending_inventory_ibm[0]=model.addConstr(Ei[0] == beginning_inventory_ibm + Pi[0] - Si[0])
ending_inventory_ibm[1]=model.addConstr(Ei[1] == Ei[0] + Pi[1] - Si[1])
ending_inventory_ibm[2]=model.addConstr(Ei[2] == Ei[1] + Pi[2] - Si[2])
ending_inventory_ibm[3]=model.addConstr(Ei[3] == Ei[2] + Pi[3] - Si[3])

# ending inventory mac
ending_inventory_mac={}
ending_inventory_mac[0]=model.addConstr(Em[0] == beginning_inventory_mac + Pm[0] - Sm[0])
ending_inventory_mac[1]=model.addConstr(Em[1] == Em[0] + Pm[1] - Sm[1])
ending_inventory_mac[2]=model.addConstr(Em[2] == Em[1] + Pm[2] - Sm[2])
ending_inventory_mac[3]=model.addConstr(Em[3] == Em[2] + Pm[3] - Sm[3])

In [ ]:
# fourth quarter ibm inventory and production constraint:
fourth_quarter_ibm={}
fourth_quarter_ibm[3]=model.addConstr(Ei[3]+Pi[3]>=24000)

# fourth quarter mac inventory and production constraint:
fourth_quarter_mac={}
fourth_quarter_mac[3]=model.addConstr(Em[3]+Pm[3]>=4800)

In [ ]:
# marketing constraint ibm 
con_marketing_ibm={}
con_marketing_ibm[0]=model.addConstr(Mi[0]>=(1/3)*Si[0])
con_marketing_ibm[1]=model.addConstr(Mi[1]>=(1/3)*Si[1])
con_marketing_ibm[2]=model.addConstr(Mi[2]>=(1/3)*Si[2])
con_marketing_ibm[3]=model.addConstr(Mi[3]>=(1/3)*Si[3])

# marketing constraint mac
con_marketing_mac={}
con_marketing_mac[0]=model.addConstr(Mm[0]>=(1/3)*Sm[0])
con_marketing_mac[1]=model.addConstr(Mm[1]>=(1/3)*Sm[1])
con_marketing_mac[2]=model.addConstr(Mm[2]>=(1/3)*Sm[2])
con_marketing_mac[3]=model.addConstr(Mm[3]>=(1/3)*Sm[3])

In [ ]:
# marketing demand constraint ibm
con_demand_marketing_ibm={}
con_demand_marketing_ibm[0]=model.addConstr(Mi[0]+Ri[0]==Si[0])
con_demand_marketing_ibm[1]=model.addConstr(Mi[1]+Ri[1]==Si[1])
con_demand_marketing_ibm[2]=model.addConstr(Mi[2]+Ri[2]==Si[2])
con_demand_marketing_ibm[3]=model.addConstr(Mi[3]+Ri[3]==Si[3])

# marketing demand constraint mac
con_demand_marketing_mac={}
con_demand_marketing_mac[0]=model.addConstr(Mm[0]+Rm[0]==Sm[0])
con_demand_marketing_mac[1]=model.addConstr(Mm[1]+Rm[1]==Sm[1])
con_demand_marketing_mac[2]=model.addConstr(Mm[2]+Rm[2]==Sm[2])
con_demand_marketing_mac[3]=model.addConstr(Mm[3]+Rm[3]==Sm[3])

In [ ]:
#max borrowing constraint
con_max_borrowing={}
con_max_borrowing[0]=model.addConstr(B[0]<=750000)
con_max_borrowing[1]=model.addConstr(B[1]<=750000)
con_max_borrowing[2]=model.addConstr(B[2]<=750000)
con_max_borrowing[3]=model.addConstr(B[3]<=750000)

In [ ]:
##cash flow each quarter
cash_flow={}
cash_flow[0]=model.addConstr(CF[0]==(price_ibm*(0.75*Si[0]+0.20*8000+0.03*7000+0.02*0.5*7000)
                                    + price_mac*(0.80*Sm[0]+0.11*1600+0.05*1400+0.04*0.5*1400)
                                    - VC_ibm*Pi[0] - VC_mac*Pm[0]             
                                    -((beginning_inventory_ibm + Ei[0])/2 * VC_ibm * 3*carry_cost_month)
                                     -((beginning_inventory_mac  + Em[0])/2 * VC_mac  * 3*carry_cost_month)
                                     - cost_advertising *(Mi[0]+Mm[0])-fixed_costs  
                                     + B[0]))

cash_flow[1]=model.addConstr(CF[1]==(price_ibm*(0.20*Si[0]+0.75*Si[1]+0.03*80000+.02*0.5*8000)
                                    + price_mac*(0.11*Sm[0]+ 0.80*Sm[1]+0.05*1600+0.04*1600*0.5)
                                     - VC_ibm*Pi[1] - VC_mac*Pm[1]
                                     - ((Ei[0] + Ei[1])/2 * VC_ibm  * 3*carry_cost_month) 
                                     - ((Em[0] + Em[1])/2 * VC_mac * 3*carry_cost_month) 
                                     - cost_advertising *(Mi[1]+Mm[1])-fixed_costs
                                     - B[0]*((1+interest_rate_LOC)**0.25)+B[1]))

cash_flow[2]=model.addConstr(CF[2]==(price_ibm*(0.03*Si[0] + 0.02*0.5*Si[0] + 0.20*Si[1]+ 0.75*Si[2])
                                    + price_mac*(0.05*Sm[0] + 0.11*Sm[1]+ 0.80*Sm[2]+ 0.04*0.5*Sm[0])
                                    - VC_ibm*Pi[2] - VC_mac*Pm[2]
                                    - ((Ei[1] + Ei[2])/2 * VC_ibm  * 3*carry_cost_month) 
                                     - ((Em[1] + Em[2])/2 * VC_mac * 3*carry_cost_month) 
                                     - cost_advertising *(Mi[2]+Mm[2])-fixed_costs
                                     - B[1]*((1+interest_rate_LOC)**0.25)+B[2]))

cash_flow[3]=model.addConstr(CF[3]==(price_ibm*(0.03*Si[1] + 0.02*0.5*Si[1] + 0.20*Si[2]+ 0.75*Si[3])
                                     +price_mac*(0.05*Sm[1] + 0.04*0.5*Sm[1] + 0.11*Sm[2]+ 0.80*Sm[3])
                                     - VC_ibm*Pi[3] - VC_mac*Pm[3]
                                     -((Ei[2]+ Ei[3])/2 * VC_ibm  * 3*carry_cost_month) 
                                     - ((Em[2] + Em[3])/2 * VC_mac * 3*carry_cost_month) 
                                     - cost_advertising *(Mi[3]+Mm[3])-fixed_costs
                                     -B[2]*((1+interest_rate_LOC)**0.25)+B[3]))

In [ ]:
# cash flow first two quarters of 1997
forward_profit={}
forward_profit[0]=model.addConstr(F[0] == ((price_ibm*(0.03*Si[2] + 0.02*0.50*Si[2]+ 0.20*Si[3]) 
                                          + price_mac*(0.05*Sm[2]  + 0.04*0.5*Sm[2]+ 0.11*Sm[3])
                                          -B[3]*((1+interest_rate_LOC)**0.25))*((1+interest_rate_bonds)**-0.25)))
forward_profit[1]=model.addConstr(F[1] == ((price_ibm*(0.03*Si[3] + 0.02*0.5*Si[3]) 
                                          + price_mac*(0.05*Sm[3] + 0.04*0.5*Sm[3]))*((1+interest_rate_bonds)**-0.5)))


In [ ]:
# non-negative Cash flows
cash_flow_non_negative={}
cash_flow_non_negative[0]=model.addConstr(CF[0]>=0)
cash_flow_non_negative[1]=model.addConstr(CF[1]>=0)
cash_flow_non_negative[2]=model.addConstr(CF[2]>=0)
cash_flow_non_negative[3]=model.addConstr(CF[3]>=0)

In [ ]:
# regular sales limited by forecasted demand IBM
regular_sales_ibm={}
regular_sales_ibm[0]=model.addConstr(Ri[0]<=9000)
regular_sales_ibm[1]=model.addConstr(Ri[1]<=10000)
regular_sales_ibm[2]=model.addConstr(Ri[2]<=11000)
regular_sales_ibm[3]=model.addConstr(Ri[3]<=12000)

# regular sales limited by forecasted demand IBM
regular_sales_mac={}
regular_sales_mac[0]=model.addConstr(Rm[0]<=1800)
regular_sales_mac[1]=model.addConstr(Rm[1]<=2000)
regular_sales_mac[2]=model.addConstr(Rm[2]<=2200)
regular_sales_mac[3]=model.addConstr(Rm[3]<=2400)


**Objective Function**

$$\max\bigg((((CF[0]\times((1+interest\_rate\_bonds)^{1/4}))+CF[1])\times((1+interest\_rate\_bonds)^{1/4}))+CF[2])\times((1+interest\_rate\_bonds)^{1/4})+CF[3]+F[0]+F[1] \bigg)$$


**Objective Function Explanation:** The objective function is ultimately adding up the cash flows from period 1 to 4. Our assumption is that at the end of the period, all positive cash flow is invested in a government security. As a result, the cash flows ends up to be zero. The following period, the principal plus the interest is received and added to the new cash flow. This is iterative until the fourth quarter where the third cash flow principal and interest is received and is added to the fourth cash flow, plus the future cash flows of the first two quarters of 1997, discounted back. We are therefore trying to find the maximum of the fourth quarter, which combined all of the cash flows of the previous quarters as well as two first quarters of 1997, to maximize the profits of that year. 

In [ ]:
# objective function
obj = ((((CF[0]*((1+interest_rate_bonds)**0.25))+CF[1])*((1+interest_rate_bonds)**0.25))+CF[2])*((1+interest_rate_bonds)**0.25)+CF[3]+F[0]+F[1]
       
model.setObjective(obj, GRB.MAXIMIZE)

In [ ]:
model.optimize()

### **Optimal Solution**

In [ ]:
base_model=model.ObjVal
print("The optimal solution will result in a profit of:", "$",model.objVal, "at the end of Q4 1996") 

In [ ]:
print("Here is a table containing key summary information regarding the optimal solution:")
Solution = Table().with_columns([
     'Quarters',[1,2,3,4],
    'Production IBM',[Pi[0].x, Pi[1].x,Pi[2].x,Pi[3].x],
    'Production Mac',[Pm[0].x, Pm[1].x,Pm[2].x,Pm[3].x],
    'Sales IBM',[Si[0].x, Si[1].x,Si[2].x,Si[3].x],
    'Sales Mac',[Sm[0].x, Sm[1].x,Sm[2].x,Sm[3].x],
    'Regular Sales IBM',[Ri[0].x, Ri[1].x,Ri[2].x,Ri[3].x],
    'Regular Sales Mac',[Rm[0].x, Rm[1].x,Rm[2].x,Rm[3].x],
    'Marketing IBM',[Mi[0].x, Mi[1].x,Mi[2].x,Mi[3].x],
    'Marketing Mac',[Mm[0].x, Mm[1].x,Mm[2].x,Mm[3].x],
    'Amount borrowed',[B[0].x, B[1].x,B[2].x,B[3].x],
    'Ending inventory IBM',[Ei[0].x, Ei[1].x,Ei[2].x,Ei[3].x],
    'Ending inventory Mac',[Em[0].x, Em[1].x,Em[2].x,Em[3].x],
    'Cash flow',[CF[0].x, CF[1].x,CF[2].x,CF[3].x],
    ])
Solution


In [ ]:
Forward = Table().with_columns([
     'Quarters',[1,2],
     '1997 Cash Flows',[F[0].x, F[1].x],
    ])
Forward

In [ ]:
plots.bar(Solution.column("Quarters")-0.1,Solution.column("Production IBM"),width=0.2,label="IBM Production")
plots.bar(Solution.column("Quarters")+0.1,Solution.column("Production Mac"),width=0.2,label="MAC Production")
plots.xticks([1,2,3,4])
plots.axis(xmax=4.5,ymax=40000)
plots.xlabel("Quarter",fontsize=15)
plots.ylabel("Production in units",fontsize=15)
plots.title("Production of adapters over the year 1996",fontsize=20)
_ =plots.legend(loc="best")

In [ ]:
plots.bar(Solution.column("Quarters")-0.1,Solution.column("Sales IBM"),width=0.2,label="IBM Sales")
plots.bar(Solution.column("Quarters")+0.1,Solution.column("Sales Mac"),width=0.2,label="MAC Sales")
plots.xticks([1,2,3,4])
plots.axis(xmax=4.5,ymax=50000)
plots.xlabel("Quarter",fontsize=15)
plots.ylabel("Sales in units",fontsize=15)
plots.title("Sales of adapters over the year 1996",fontsize=20)
_ =plots.legend(loc="upper right")

In [ ]:
plots.bar(Solution.column("Quarters")-0.1,Solution.column("Marketing IBM"),width=0.2,label="IBM Marketing")
plots.bar(Solution.column("Quarters")+0.1,Solution.column("Marketing Mac"),width=0.2,label="MAC Marketing")
plots.xticks([1,2,3,4])
plots.axis(xmax=4.5,ymax=50000)
plots.xlabel("Quarter",fontsize=15)
plots.ylabel("Marketing in units of additional adapter units sold due to marketing activities",fontsize=15)
plots.title("Marketing of adapters over the year 1996",fontsize=20)
_ =plots.legend(loc="best")

In [ ]:
plots.bar(Solution.column("Quarters")-0.1,Solution.column("Amount borrowed"),width=0.2,label="Amount Borrowed")
plots.xticks([1,2,3,4])
plots.axis(xmax=4.5,ymax=800000)
plots.xlabel("Quarter",fontsize=15)
plots.ylabel("Amount borrowed from the line of credit in units of dollars",fontsize=15)
plots.title("Amount over the year 1996",fontsize=20)
_ =plots.legend(loc="best")

In [ ]:
plots.bar(Forward.column("Quarters")-0.1,Forward.column("1997 Cash Flows"),width=0.2,label="Cash Flow")
plots.xticks([1,2])
plots.axis(xmax=4.5,ymax=3.5e+06)
plots.xlabel("Quarter",fontsize=15)
plots.ylabel("Cash flow at the end of the quarter in units of dollars",fontsize=15)
plots.title("Cash flow over the year 1997",fontsize=20)
_ =plots.legend(loc="upper right")

In [ ]:
plots.bar(Solution.column("Quarters")-0.1,Solution.column("Cash flow"),width=0.2,label="Cash Flow")
plots.xticks([1,2,3,4])
plots.axis(xmax=4.5,ymax=4.0e+06)
plots.xlabel("Quarter",fontsize=15)
plots.ylabel("Cash flow at the end of the quarter in units of dollars",fontsize=15)
plots.title("Cash flow over the year 1996",fontsize=20)
_ =plots.legend(loc="upper right")

In [ ]:
plots.bar(Solution.column("Quarters")-0.1,Solution.column("Ending inventory IBM"),width=0.2,label="IBM Ending Inventory")
plots.bar(Solution.column("Quarters")+0.1,Solution.column("Ending inventory Mac"),width=0.2,label="MAC Ending Inventory")
plots.xticks([1,2,3,4])
plots.axis(xmax=4.5,ymax=10000)
plots.xlabel("Quarter",fontsize=15)
plots.ylabel("Ending inventory in units of adapters",fontsize=15)
plots.title("Ending Inventory over the year 1996",fontsize=20)
_ =plots.legend(loc="best")

## **Key findings from the optimal solution:**
$ 13031411.858522063
* No borrowing
*   Regular Sales maxed out in Q1-Q4
*   Marketing Spend is Very Important to Offset Relatively Low Fixed Costs with Additional Sales
*   Marketing Sales far outweigh Regular Sales
* Ending Inventories of 0 for both adapters in Q1 and Q4 
* Production capacity maxed out for both adapters in Q1, Q2, Q3
* Production under capacity for both adapters in Q4
*Ratio of Marketing Sales between both adapters is more similar than Ratio between for Regular Products
*Q3 has smallest net cash flow
*Q1 has highest cash flow 


# **Section III: Sensitivity Analysis**

Questions 7  is present in this report as a sample. All of the sensitivity analyses are present in separate .ipynb files submitted with this report. 

### **Sensitivity Question 7**

In [ ]:
rate=0.24

model2 = Model('sensitivity analysis') # object

#initilizing empty dictionaries
Pi = {}  
Pm = {} 
Si = {} 
Sm = {} 
Mi = {} 
Mm = {} 
B = {} 
Ei = {}
Em = {} 
CF={}
F= {}
Ri={}
Rm={}

# Create decisions variable
for i in np.arange(4):  
    Pi[i] = model2.addVar(lb=0)
    Pm[i] = model2.addVar(lb=0)
    Si[i] = model2.addVar(lb=0)
    Sm[i] = model2.addVar(lb=0)
    Mi[i] = model2.addVar(lb=0)
    Mm[i] = model2.addVar(lb=0)
    B[i] = model2.addVar(lb=0)
    CF[i] = model2.addVar(lb=0)
    Ei[i] = model2.addVar(lb=0)
    Em[i] = model2.addVar(lb=0)
    Ri[i] = model2.addVar(lb=0)
    Rm[i] = model2.addVar(lb=0)

for i in np.arange(2):    
    F[i] = model2.addVar(lb=0)
    # By default Gurobi assumes that the lower bound of a variable lb=0, not minus infinity.

# constant parameters  
price_ibm = 175
price_mac = 200
VC_ibm = 100
VC_mac = 110
beginning_inventory_ibm = 10000
beginning_inventory_mac = 5000
cost_advertising = 10
interest_rate_LOC = rate
interest_rate_bonds = 0.06
fixed_costs = 250000
carry_cost_month = 0.01

# creating constraint
# maximum production ibm constraint
con_max_production_ibm={}
con_max_production_ibm[0]=model2.addConstr(Pi[0]<=28800)
con_max_production_ibm[1]=model2.addConstr(Pi[1]<=28800)
con_max_production_ibm[2]=model2.addConstr(Pi[2]<=28800)
con_max_production_ibm[3]=model2.addConstr(Pi[3]<=31680)

# maximum production mac constraint
con_max_production_mac={}
con_max_production_mac[0]=model2.addConstr(Pm[0]<=9600)
con_max_production_mac[1]=model2.addConstr(Pm[1]<=9600)
con_max_production_mac[2]=model2.addConstr(Pm[2]<=9600)
con_max_production_mac[3]=model2.addConstr(Pm[3]<=10560)

# minimum production ibm constraint
con_min_production_ibm={}
con_min_production_ibm[0]=model2.addConstr(Pi[0]>=2400)
con_min_production_ibm[1]=model2.addConstr(Pi[1]>=2400)
con_min_production_ibm[2]=model2.addConstr(Pi[2]>=2400)
con_min_production_ibm[3]=model2.addConstr(Pi[3]>=2400)

# minimum production mac constraint
con_min_production_mac={}
con_min_production_mac[0]=model2.addConstr(Pm[0]>=1800)
con_min_production_mac[1]=model2.addConstr(Pm[1]>=1800)
con_min_production_mac[2]=model2.addConstr(Pm[2]>=1800)
con_min_production_mac[3]=model2.addConstr(Pm[3]>=1800)

# inventory management constraint ibm
estimated_sale_ibm={}
estimated_sale_ibm[0]=model2.addConstr(Pi[0] +Ei[0] >= 1.1*Si[1])
estimated_sale_ibm[1]=model2.addConstr(Pi[1] +Ei[1] >= 1.1*Si[2])
estimated_sale_ibm[2]=model2.addConstr(Pi[2] +Ei[2] >= 1.1*Si[3])

# inventory management constraint mac
estimated_sale_mac={}
estimated_sale_mac[0]=model2.addConstr(Pm[0] +Em[0] >= 1.1*Sm[1])
estimated_sale_mac[1]=model2.addConstr(Pm[1] +Em[1] >= 1.1*Sm[2])
estimated_sale_mac[2]=model2.addConstr(Pm[2] +Em[2] >= 1.1*Sm[3])

# ending inventory ibm 
ending_inventory_ibm={}
ending_inventory_ibm[0]=model2.addConstr(Ei[0] == beginning_inventory_ibm + Pi[0] - Si[0])
ending_inventory_ibm[1]=model2.addConstr(Ei[1] == Ei[0] + Pi[1] - Si[1])
ending_inventory_ibm[2]=model2.addConstr(Ei[2] == Ei[1] + Pi[2] - Si[2])
ending_inventory_ibm[3]=model2.addConstr(Ei[3] == Ei[2] + Pi[3] - Si[3])

# ending inventory mac
ending_inventory_mac={}
ending_inventory_mac[0]=model2.addConstr(Em[0] == beginning_inventory_mac + Pm[0] - Sm[0])
ending_inventory_mac[1]=model2.addConstr(Em[1] == Em[0] + Pm[1] - Sm[1])
ending_inventory_mac[2]=model2.addConstr(Em[2] == Em[1] + Pm[2] - Sm[2])
ending_inventory_mac[3]=model2.addConstr(Em[3] == Em[2] + Pm[3] - Sm[3])

# fourth quarter ibm inventory and production constraint:
fourth_quarter_ibm={}
fourth_quarter_ibm[3]=model2.addConstr(Ei[3]+Pi[3]>=24000)

# fourth quarter mac inventory and production constraint:
fourth_quarter_mac={}
fourth_quarter_mac[3]=model2.addConstr(Em[3]+Pm[3]>=4800)

# marketing constraint ibm 
con_marketing_ibm={}
con_marketing_ibm[0]=model2.addConstr(Mi[0]>=(1/3)*Si[0])
con_marketing_ibm[1]=model2.addConstr(Mi[1]>=(1/3)*Si[1])
con_marketing_ibm[2]=model2.addConstr(Mi[2]>=(1/3)*Si[2])
con_marketing_ibm[3]=model2.addConstr(Mi[3]>=(1/3)*Si[3])

# marketing constraint mac
con_marketing_mac={}
con_marketing_mac[0]=model2.addConstr(Mm[0]>=(1/3)*Sm[0])
con_marketing_mac[1]=model2.addConstr(Mm[1]>=(1/3)*Sm[1])
con_marketing_mac[2]=model2.addConstr(Mm[2]>=(1/3)*Sm[2])
con_marketing_mac[3]=model2.addConstr(Mm[3]>=(1/3)*Sm[3])

# marketing demand constraint ibm
con_demand_marketing_ibm={}
con_demand_marketing_ibm[0]=model2.addConstr(Mi[0]+Ri[0]==Si[0])
con_demand_marketing_ibm[1]=model2.addConstr(Mi[1]+Ri[1]==Si[1])
con_demand_marketing_ibm[2]=model2.addConstr(Mi[2]+Ri[2]==Si[2])
con_demand_marketing_ibm[3]=model2.addConstr(Mi[3]+Ri[3]==Si[3])

# marketing demand constraint mac
con_demand_marketing_mac={}
con_demand_marketing_mac[0]=model2.addConstr(Mm[0]+Rm[0]==Sm[0])
con_demand_marketing_mac[1]=model2.addConstr(Mm[1]+Rm[1]==Sm[1])
con_demand_marketing_mac[2]=model2.addConstr(Mm[2]+Rm[2]==Sm[2])
con_demand_marketing_mac[3]=model2.addConstr(Mm[3]+Rm[3]==Sm[3])

#max borrowing constraint
con_max_borrowing={}
con_max_borrowing[0]=model2.addConstr(B[0]<=750000)
con_max_borrowing[1]=model2.addConstr(B[1]<=750000)
con_max_borrowing[2]=model2.addConstr(B[2]<=750000)
con_max_borrowing[3]=model2.addConstr(B[3]<=750000)

##cash flow each quarter
cash_flow={}
cash_flow[0]=model2.addConstr(CF[0]==(price_ibm*(0.75*Si[0]+0.20*8000+0.03*7000+0.02*0.5*7000)
                                    + price_mac*(0.80*Sm[0]+0.11*1600+0.05*1400+0.04*0.5*1400)
                                    - VC_ibm*Pi[0] - VC_mac*Pm[0]             
                                    -((beginning_inventory_ibm + Ei[0])/2 * VC_ibm * 3*carry_cost_month)
                                    -((beginning_inventory_mac  + Em[0])/2 * VC_mac  * 3*carry_cost_month)
                                    - cost_advertising *(Mi[0]+Mm[0])-fixed_costs  
                                    + B[0]))

cash_flow[1]=model2.addConstr(CF[1]==(price_ibm*(0.20*Si[0]+0.75*Si[1]+0.03*80000+.02*0.5*8000)
                                    + price_mac*(0.11*Sm[0]+ 0.80*Sm[1]+0.05*1600+0.04*1600*0.5)
                                    - VC_ibm*Pi[1] - VC_mac*Pm[1]
                                    - ((Ei[0] + Ei[1])/2 * VC_ibm  * 3*carry_cost_month) 
                                    - ((Em[0] + Em[1])/2 * VC_mac * 3*carry_cost_month) 
                                    - cost_advertising *(Mi[1]+Mm[1])-fixed_costs
                                    - B[0]*((1+interest_rate_LOC)**0.25)+B[1]))

cash_flow[2]=model2.addConstr(CF[2]==(price_ibm*(0.03*Si[0] + 0.02*0.5*Si[0] + 0.20*Si[1]+ 0.75*Si[2])
                                    + price_mac*(0.05*Sm[0] + 0.11*Sm[1]+ 0.80*Sm[2]+ 0.04*0.5*Sm[0])
                                    - VC_ibm*Pi[2] - VC_mac*Pm[2]
                                    - ((Ei[1] + Ei[2])/2 * VC_ibm  * 3*carry_cost_month) 
                                    - ((Em[1] + Em[2])/2 * VC_mac * 3*carry_cost_month) 
                                    - cost_advertising *(Mi[2]+Mm[2])-fixed_costs
                                    - B[1]*((1+interest_rate_LOC)**0.25)+B[2]))

cash_flow[3]=model2.addConstr(CF[3]==(price_ibm*(0.03*Si[1] + 0.02*0.5*Si[1] + 0.20*Si[2]+ 0.75*Si[3])
                                    +price_mac*(0.05*Sm[1] + 0.04*0.5*Sm[1] + 0.11*Sm[2]+ 0.80*Sm[3])
                                    - VC_ibm*Pi[3] - VC_mac*Pm[3]
                                    -((Ei[2]+ Ei[3])/2 * VC_ibm  * 3*carry_cost_month) 
                                    - ((Em[2] + Em[3])/2 * VC_mac * 3*carry_cost_month) 
                                    - cost_advertising *(Mi[3]+Mm[3])-fixed_costs
                                    -B[2]*((1+interest_rate_LOC)**0.25)+B[3]))


# cash flow first two quarters of 1997
forward_profit={}
forward_profit[0]=model2.addConstr(F[0] == ((price_ibm*(0.03*Si[2] + 0.02*0.50*Si[2]+ 0.20*Si[3]) 
                                          + price_mac*(0.05*Sm[2]  + 0.04*0.5*Sm[2]+ 0.11*Sm[3])
                                          -B[3]*((1+interest_rate_LOC)**0.25))*((1+interest_rate_bonds)**-0.25)))
forward_profit[1]=model2.addConstr(F[1] == ((price_ibm*(0.03*Si[3] + 0.02*0.5*Si[3]) 
                                          + price_mac*(0.05*Sm[3] + 0.04*0.5*Sm[3]))*((1+interest_rate_bonds)**-0.5)))



# non-negative Cash flows
cash_flow_non_negative={}
cash_flow_non_negative[0]=model2.addConstr(CF[0]>=0)
cash_flow_non_negative[1]=model2.addConstr(CF[1]>=0)
cash_flow_non_negative[2]=model2.addConstr(CF[2]>=0)
cash_flow_non_negative[3]=model2.addConstr(CF[3]>=0)

# regular sales limited be forecasted demand IBM
regular_sales_ibm={}
regular_sales_ibm[0]=model2.addConstr(Ri[0]<=9000)
regular_sales_ibm[1]=model2.addConstr(Ri[1]<=10000)
regular_sales_ibm[2]=model2.addConstr(Ri[2]<=11000)
regular_sales_ibm[3]=model2.addConstr(Ri[3]<=12000)

# regular sales limited be forecasted demand IBM
regular_sales_mac={}
regular_sales_mac[0]=model2.addConstr(Rm[0]<=1800)
regular_sales_mac[1]=model2.addConstr(Rm[1]<=2000)
regular_sales_mac[2]=model2.addConstr(Rm[2]<=2200)
regular_sales_mac[3]=model2.addConstr(Rm[3]<=2400)


# objective function
obj = ((((CF[0]*((1+interest_rate_bonds)**0.25))+CF[1])*((1+interest_rate_bonds)**0.25))+CF[2])*((1+interest_rate_bonds)**0.25)+CF[3]+F[0]+F[1]
    
model2.setObjective(obj, GRB.MAXIMIZE)

model2.optimize()

In [ ]:

print("\nThe optimal solution with a line of credit interest rate of", rate, "will result in a profit of:", "$", model2.objVal, "at the end of Q4 1996") 

In [ ]:
model2.ObjVal==model.objVal
print("It is", model2.ObjVal==model.objVal, "that the solution to the objective function (profit) remains the same with the change in interest rate" )

In [ ]:
print("Here is a table containing key summary information regarding the optimal solution in the sensitivity analysis, changing line of credit interest rate to 24%:")
Sensitivity_solution = Table().with_columns([
    'Quarters',[1,2,3,4],
    'Production IBM',[Pi[0].x, Pi[1].x,Pi[2].x,Pi[3].x],
    'Production Mac',[Pm[0].x, Pm[1].x,Pm[2].x,Pm[3].x],
    'Sales IBM',[Si[0].x, Si[1].x,Si[2].x,Si[3].x],
    'Sales Mac',[Sm[0].x, Sm[1].x,Sm[2].x,Sm[3].x],
    'Regular Sales IBM',[Ri[0].x, Ri[1].x,Ri[2].x,Ri[3].x],
    'Regular Sales Mac',[Rm[0].x, Rm[1].x,Rm[2].x,Rm[3].x],
    'Marketing IBM',[Mi[0].x, Mi[1].x,Mi[2].x,Mi[3].x],
    'Marketing Mac',[Mm[0].x, Mm[1].x,Mm[2].x,Mm[3].x],
    'Amount borrowed',[B[0].x, B[1].x,B[2].x,B[3].x],
    'Ending inventory IBM',[Ei[0].x, Ei[1].x,Ei[2].x,Ei[3].x],
    'Ending inventory Mac',[Em[0].x, Em[1].x,Em[2].x,Em[3].x],
    'Cash flow',[CF[0].x, CF[1].x,CF[2].x,CF[3].x],
    ])
Sensitivity_solution


In [ ]:
Forward = Table().with_columns([
     'Quarters',[1,2],
     '1997 Cash Flows',[F[0].x, F[1].x],
    ])
Forward

In [ ]:
print("Here is a table containing the possible changes to the line of credit and profit as a result of changing the line of credit interest rate:")
Sensitivity_solution_table = Table().with_columns([
    '',["-", "-", "Difference"],
    'Line of Credit interest rate',[0.16, 0.24, (0.24-0.16)],
    'Amount borrowed Q1',[0, B[0].x, (B[0].x-0)],
    'Amount borrowed Q2',[0, B[1].x, (B[1].x-0)],
    'Amount borrowed Q3',[0, B[2].x, (B[2].x-0)],
    'Amount borrowed Q4',[0, B[3].x, (B[3].x-0)],
    'Objective function profit',[original_solution, model2.objVal, (model2.objVal-original_solution)],
    ])

Sensitivity_solution_table


In [ ]:
plots.bar(Sensitivity_solution_table.column("Line of Credit interest rate")[0:2],Sensitivity_solution_table.column("Objective function profit")[0:2],width=0.02,label="Amount Borrowed")
plots.xticks([0.05,0.1,0.15,0.2, 0.25])
plots.axis(xmax=0.3,ymax=1.30314e+07)
plots.xlabel("Interest Rate",fontsize=15)
plots.ylabel("Profit at the end of Q4 1996 in units of dollars",fontsize=15)
plots.title("Amount over the year 1996",fontsize=20)
_ =plots.legend(loc="lower left")

The change from an interest rate on the line of credit of 0.16 to an interest rate of 0.24 by going with a smaller bank is ultimately not affecting the ending profit at the end of Q4 of 1996. The change in the line of credit interest rate results in a change to the objective function, since this rate is used in the 1996 and 1997 cash flows, all present indirectly as coefficients in the objective function. However, since no boring is being done at all, the increase in this interest rate does not affect the profit. The borrowing portion of the equation results in zero, so changing the line of credit interest rate marginally will not impact these  quarters. 

*The change in bank does not affect the profit generated at the end of Q4.* 


### **Sensitivity Question 11(a)**

The addition to the line of credit by 1000000 is not affecting the ending profit at the end of Q4 of 1996. The increase in the line of credit results in a change to the right hand side of the borrowing constraints. However, since no borrowing is being done at all, the increase in this line of credit quantity is not affecting the profit. To mazimize profit, the company could have borrowed up until $750 000 and did not; it is not a binding constraint. In Q1, Q2, Q4 of 1996, the borrowing coefficients ended up being zero, and since no borrowing is done, changing the line of credit will not impact these quarters. 

*As a result of the profit staying the same, the additional 1M line of credit is not needed, as borrowing could have taken place in the initial solution and was not.* 


In [ ]:
IPython.display.Image('https://raw.githubusercontent.com/tyradavid/MGSC404-Lab01/master/Midterm%20Photos/Question11a.PNG') 

### **Sensitivity Question 3(a)**

The decrease in price to 173 dollars for IBM adapters and 198 dollars for Macintosh adapters makes a difference in the result of the objective function. The maximum profit is now approximately 12,689,665.92 dollars, which is a decrease of approximately 341,745.94 dollars from the original solution. In more detail, there are subtle differences between the tables of the original solution and this new solution, all of which pertaining to cash flows. With these new prices, cash flows in quarters 1, 2, 3, and 4 of 1996 decrease by approximately 85,870, 77,150, 75,620, and 75,350, dollars respectively. Furthermore, the 1997 cash flows of quarters 1, and 2 also decrease by approximately 16,990 and 3,541 dollars respectively. The decrease of cash flow in each quarter and the decrease in overall profit makes sense, since American Office Systems is making less money with each sale of adapters due to the decrease in price.   Please refer to sensitivity analysis 3a notebook file for full model and findings.

### **Sensitivity Question 3(b)**

The decrease in price to 172 dollars for IBM adapters and 197 dollars for Macintosh adapters makes a difference in the result of the objective function. The profit is now approximately 12,518,792.95 dollars, which is a decrease of approximately 512,618.91 dollars from the original solution. In more detail, there are subtle differences between the tables of the original solution and this new solution, all of which pertaining to cash flows. With these new prices, cash flows in quarters 1, 2, 3, and 4 of 1996 decrease by approximately 128,800, 115,730, 113,420, 113,030 dollars respectively. Furthermore, the 1997 cash flows of quarters 1, and 2 also decrease by approximately 25,490 and 5,311 dollars respectively. The decrease of cash flow in each quarter and the decrease in overall profit makes sense, since American Office Systems is making less money with each sale of adapters due to the decrease in price. Furthermore, it makes sense that the decreases in cash flows are more dramatic for this sensitivity analysis compared to the previous one, since the price of the adapters is even lower. Please refer to sensitivity analysis 3b notebook file for full model and findings.

### **Sensitivity Question 3(c)**

The increase in price to 180 dollars for IBM adapters and 205 dollars for Macintosh adapters makes a difference in the result of the objective function. The maximum profit is now approximately 13,885,776.71 dollars, which is an increase of approximately 854,364.85 dollars from the original solution. In more detail, there are subtle differences between the tables of the original solution and this new solution, all of which pertaining to cash flows. With these new prices, cash flows in quarters 1, 2, 3, and 4 of 1996 increase by approximately 214,670, 192,880, 189,040, and 188,380 dollars respectively. Furthermore, the 1997 cash flows of quarters 1, and 2 also increase by approximately 42,480 and 8,851 dollars respectively. The increase of cash flow in each quarter and the increase in overall profit makes sense, since American Office Systems is making more money with each sale of adapters, due to the increase in price.   Please refer to sensitivity analysis 3c notebook file for full model and findings.

### **Sensitivity Question 4(a)**

The increase in production cost to 102 dollars for IBM adapters and 112 dollars for Macintosh adapters makes a difference in the result of the objective function. The maximum profit is now approximately 12724143.63 dollars, which is a decrease of approximately 307,268.23 dollars from the original solution. In more detail, there are subtle differences between the tables of the original solution and this new solution, all of which pertaining to cash flows. With these new production costs, cash flows in quarters 1, 2, 3, and 4 of 1996 decrease by approximately 77,250, 76,910, 77,020, and 69,240 dollars respectively. Interestingly, the 1997 cash flows do not change with the decrease in production costs. The decrease of cash flow in each quarter and the decrease in overall profit makes sense, since American Office Systems is having to spend more money producing each adapter, due to the increase in production costs. Please refer to sensitivity analysis 4a notebook file for full model and findings.

### **Sensitivity Question 4(b)**

The increase in production cost to 105 dollars for IBM adapters and 115 dollars for Macintosh adapters makes a difference in the result of the objective function. The maximum profit is now approximately 12,263,241.29 dollars, which is a decrease of approximately 768,170.57 dollars from the original solution. In more detail, there are subtle differences between the tables of the original solution and this new solution, all of which pertaining to cash flows. With these new production costs, cash flows in quarters 1, 2, 3, and 4 of 1996 decrease by approximately 193,130, 192,260, 192,550, and 173,100 dollars respectively. As in the previous sensitivity analysis, the 1997 cash flows do not change with the decrease in production costs. The decrease of cash flow in each quarter and the decrease in overall profit makes sense, since American Office Systems is having to spend more money producing each adapter due to the increase in production costs. Furthermore, it makes sense that the decreases in cash flows are more dramatic for this sensitivity analysis compared to the previous one since the production cost of the adapters is even higher. Please refer to sensitivity analysis 4a notebook file for full model and findings.

## **Section IV: Original Sensitivity Analysis**

## **Original Sensitivity Analysis 1: Work Hours (16 to 20 Operating Hours Daily)**

Production maximum capacities constraints are all based heavily on the work hours the company is able to operate with. Work hours are important since their maximum marketing sales are greatly limited to the amount by production capacity, as it influences last quarter’s ending inventory and current production. Three quarters (1,2,4) were able to reach maximum production capacities. This means the company is potentially losing extra marketing sales that enable them to offset further their fixed costs. Thus, losing any possible sales due to insufficient production to meet extra demand created by marketing is a concern.

Since labour and associated costs are already included in the product variable costs, it is reasonable for the company to consider increasing work hours. Normally, if a company expects more sales in quarter four (Christmas period), they should produce more in quarter three and quarter four, and have longer operating hours in the factory. As long as machines are able to tolerate greater usage, all-day operations (22-24 hours) can also be considered, but this leaves more risk with less time for maintenance.

Thus, since the company currently has two shifts daily of 8 hours (16 hours daily operating hours), increasing both work shifts to **10 hours for every quarter (20 daily operating hours a day)** is realistic in this 1990s work environment and reasonable enough to allow time for machine maintenance. Less than 8 hours is unrealistic, which is why settling for 10 hours is the most feasible if the company needs the same hours for each quarter. Decreasing would only lessen the company's profits.

----

### New Constraints based on 20 work hours per day instead of 16

#### **Maximum Production IBM Constraint**
Q1-3: Pi[0-2]<= 30*20*20*3 < = 36 000; 

Q4: Pi[3]<= 1.1*30*20*20*3 <= 39 600

con_max_production_ibm={}

con_max_production_ibm[0]=model2.addConstr(Pi[0]<= 36000)

con_max_production_ibm[1]=model2.addConstr(Pi[1]<= 36000)

con_max_production_ibm[2]=model2.addConstr(Pi[2]<= 36000)

con_max_production_ibm[3]=model2.addConstr(Pi[3]<= 39600)

#### **Maximum Production Mac Constraint**
Q1-3: Pm[0-2]<= 10*20*20*3 <= 12 000;
 
Q4: Pm[3] <= 1.1*10*20*20*3 <= 13200

con_max_production_mac={}

con_max_production_mac[0]=model2.addConstr(Pm[0]<= 12000)

con_max_production_mac[1]=model2.addConstr(Pm[1]<= 12000)

con_max_production_mac[2]=model2.addConstr(Pm[2]<= 12000)

con_max_production_mac[3]=model2.addConstr(Pm[3]<= 13200)

----

Thus, all that was changed where the maximum production capacity constraints per product each quarter, which are all increased **by 25% (20h/16h).** By changing the work shifts to **10 hours (20 hours daily)**, the company can expect a total profit of approximately **$15,538,152.29**, which is a: 

**$2,506,740.43** increase in total profit 

### **1996 Production Year Solution with 20-Work Hours**

In [ ]:
IPython.display.Image('https://raw.githubusercontent.com/tyradavid/MGSC404-Lab01/master/Midterm%20Photos/Solution%20-%2020H%20Work%20Days.JPG') #Solution - 20H Work Days   updated

In [ ]:
IPython.display.Image("https://raw.githubusercontent.com/tyradavid/MGSC404-Lab01/master/Midterm%20Photos/1997%20-%20Solution%20-%20Work%2020%20H.JPG") # 1997 - Solution - Work 20 H updated

This is the new optimal solution for 20 work hours. The cash flows are all higher for each quarter.

### **Yearly Differences - 20 and 16 Daily Production Hour Comparison**

In [ ]:
IPython.display.Image('https://raw.githubusercontent.com/tyradavid/MGSC404-Lab01/master/Midterm%20Photos/Differences%20-%2020H%20to%2016H%20Year.JPG')       #Differences - 20H to 16H Year          switch thanks 

As is clearly shown, the company can sell many more units through Marketing advertisements than before, which is the only cause for increased profits since regular sales were already maxed out with 16 hours.

### **Sensitivity Analysis - RHS Upper Bound for Work Hours (original solution)**

In [ ]:
IPython.display.Image("https://raw.githubusercontent.com/tyradavid/MGSC404-Lab01/master/Midterm%20Photos/Work%20Hours%20Limit%20%20-%20Code.JPG") #Work Hours Limit  - Code     updated

The reasons for the increased profits and optimal solution change are that the increase in work hours surpasses the allowable increase for right-hand side constraints regarding production (found by finding maximum value of production constraint allowed without changing objective function value and then dividing by other factors which remain the same to isolate for Daily Work Hours). Since it was previously at 16 hours, we can see the max value was surpassed by 20 work hours for all but two RHS production constraints (Mac and IBM Q4), which is why far greater profits can be achieved with more production and marketing sales.

However, even though it may be unrealistic, if the work hours are variable enough, they could have been decided upon as decision variables for each quarter to maximise profits (if within reasonable constraints). It would be interesting to see if the company can also secure an extra one million dollars for the credit line, which would enable further production, and combined with extra work hours, would present a feasonable alternative situation with greatly increased profitability (assuming marketing spend does not reach its limits in creating new demand). 

## **Original Sensitivity Analysis 2: Marketing Costs (from 10 to $15 marketing ad cost/add. unit sold)**

The marketing sales performance expectations are first done by an external advertising agency and reviewed by Ms. Williams. These forecasts can be wrong since they do not reflect actual electronics industry expertise. Usually, simple advertisements perform less well than sales promotions (discounts, coupons) for increasing electronic sales (shopping products), especially in the 1990s. However, admittedly, the practical nature of the product lends itself better to advertisements that highlight its use and the consumer need.

In any case, marketing for both adapters has never been done before (according to case), thus the ratio **($10 marketing per extra sale)** may not hold true. As marketing is an extremely important component for this model’s profitability, and as it is easy to overestimate the effect of introductory marketing on a new product, it is better to consider for caution that this ratio will not materialise and that additional spending will be required to sell an additional unit (especially considering the lack of past experience doing campaigns for these products). 

To offset the risk of imprecise marketing assumptions, **a model where 15 and not $10 (50% increase in marketing cost for conversion)** in marketing ads must be spent per additional marketing sale will be looked at for prudence.

To change this, all that was needed was to run the model by changing the advertising cost constant to :

**cost_advertising = 15**

Thus, all that was changed where the max production capacity constraints per product each quarter which are all increased **by 25% (20h/16h).** By changing the work shifts to **10 hours (20 hours daily)**, the company can expect a total profit of **$12,444,870.65**, thus a :

**-586,541.21$** decrease in total profit 


### **1996 Production Year Solution with Marketing at 15$**

In [ ]:
IPython.display.Image("https://raw.githubusercontent.com/tyradavid/MGSC404-Lab01/master/Midterm%20Photos/Solution%20-%20Marketing%2015%24.JPG") #Solution - Marketing 15$  updated

In [ ]:
IPython.display.Image("https://raw.githubusercontent.com/tyradavid/MGSC404-Lab01/master/Midterm%20Photos/1997%20-%20Solution%20-%20Marketing.JPG") #1997 - Solution - Marketing updated

As we can clearly see, the solution in terms of decisions is the same. Only the cash flows will change.

### **Cash Flow Yearly Differences - Marketing Costs 15 to 10 per add. unit Comparison**

In [ ]:
IPython.display.Image('https://raw.githubusercontent.com/tyradavid/MGSC404-Lab01/master/Midterm%20Photos/Differences%20-%20Marketing%2015%20to%2010.JPG?fbclid=IwAR1-9_QGR7vZUmKorXjXHlYKyTyDBJi1xltXtQGf1t2nxuwdYKRd-IfcX-Y')

After analysing the effect, the optimal solution does not change, only the objective function value is greatly reduced due to the greater cost of marketing conversion. The reason the decision variables do not change is that the allowable increase is greater for all Marketing decision variables than the current increase of only 5$ per unit sold through ads, as seen in the sensitivity below.

In [ ]:
IPython.display.Image('https://raw.githubusercontent.com/tyradavid/MGSC404-Lab01/master/Midterm%20Photos/Marketing%20-%20Allowable%20Code.JPG') #Marketing - Allowable Code    updated

# **Section V: Conclusion**

A linear optimization model was performed to maximize the profit of American Office Systems for the year of 1996. Our model took into account numerous constraints, which represented the many barriers that the company encountered while trying to maximize their profit. While analyzing the company, we noticed some factors that could have limited the profit of American Office Systems, and we suggest these factors be improved in order to reach a higher profit. First, there were serval limiting inventory constraints, which were designed to either provide a smooth transition to the year 1997, or to provide a safety cushion to help prevent shortages. We found these constraints to be very conservative, but we nevertheless recognize their importance. Therefore, we suggest making these constraints more aggressive by proving less of a smooth transition to 1997, or to provide less of a safety cushion, in hopes of ultimately increasing profit. Furthermore, we recommend that the left-over cash in each quarter not be reinvested in government securities. Although these securities will generate cash flow overtime, reinvesting left over cash back into the company could be more profitable. This would allow the left-over cash to be invested in research and development for example, to perhaps increase the maximum number of adapters that are able to be produced every hour, and improve other similar factors that are limiting profit. 

## Author Contributions

All three authors formulated the LP. Jonathan and Tyra did the LP coding. We all solved the LP and prepared the report. Tyra did sensitivity analysis 7 and 11a, Jonathan did sensitivity analysis 3 and 4, and Julien did the two original sensitivity analyses. Jonathan wrote the introduction, conclusion and the constraint explanations. Tyra and Julien collaborated on the objective function. 


<img src="les_miserables.jpg" style="width: 600px;"/>